In [1]:
# Import dependencies
import pandas as pd

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

# Read the CSV and Perform Basic Data Cleaning

In [2]:
# Read in data
df = pd.read_csv("exoplanet_data.csv")

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [3]:
print(df["koi_disposition"].unique())

['CONFIRMED' 'FALSE POSITIVE' 'CANDIDATE']


# Select features (columns)

In [4]:
X = df.iloc[:,1:]
y = df.iloc[:,0]

In [5]:
# Search for top 10 features according to feature importances
from sklearn.ensemble import ExtraTreesClassifier

model = ExtraTreesClassifier()
model.fit(X,y)
model.feature_importances_

array([0.13489983, 0.15982755, 0.13514723, 0.04561877, 0.0174429 ,
       0.01513379, 0.01386962, 0.01391525, 0.01696445, 0.01963672,
       0.01107878, 0.01014871, 0.00988583, 0.01659107, 0.029973  ,
       0.02166969, 0.01923625, 0.01043878, 0.01098889, 0.02576717,
       0.00911125, 0.01151998, 0.01888811, 0.00736413, 0.00876376,
       0.0103208 , 0.02948236, 0.01269665, 0.01002918, 0.02995075,
       0.01535486, 0.01117035, 0.0091051 , 0.01562803, 0.01027141,
       0.01104455, 0.00925138, 0.01007855, 0.01100872, 0.0107258 ])

In [6]:
feat_imp = pd.Series(model.feature_importances_, index=X.columns).nlargest(10)
feat_imp

koi_fpflag_ss        0.159828
koi_fpflag_co        0.135147
koi_fpflag_nt        0.134900
koi_fpflag_ec        0.045619
koi_duration_err1    0.029973
koi_steff_err1       0.029951
koi_model_snr        0.029482
koi_prad             0.025767
koi_duration_err2    0.021670
koi_time0bk_err2     0.019637
dtype: float64

In [7]:
# Set features based on feature importances
X = df[feat_imp.index]

# Use koi_disposition for y values
y = df['koi_disposition']

# Create a Train Test Split

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

# Pre-processing

Scale the data using the MinMaxScaler

In [9]:
from sklearn.preprocessing import MinMaxScaler

# Create scaler object
X_scaler = MinMaxScaler().fit(X_train)

# Scale training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model
Using a logistic regression


In [10]:
from sklearn.linear_model import LogisticRegression

# Initialize model
model = LogisticRegression()

# Train the model
model.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
# Score the model
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.8273888994850277
Testing Data Score: 0.8249427917620137


# Predict

In [12]:
predictions = model.predict(X_test_scaled)
df_pred = pd.DataFrame({"Actual":y_test, "Predicted":predictions}) 
df_pred.head()

,Actual,Predicted
3040,CANDIDATE,CONFIRMED
919,FALSE POSITIVE,FALSE POSITIVE
4307,FALSE POSITIVE,FALSE POSITIVE
5844,FALSE POSITIVE,FALSE POSITIVE
6734,FALSE POSITIVE,FALSE POSITIVE


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [13]:
from sklearn.model_selection import GridSearchCV
import numpy as np

# Create the GridSearchCV model
param_grid = {'C':np.logspace(-4, 4, 20),
             'penalty':['l1','l2']}

grid = GridSearchCV(model, param_grid, verbose=3)

In [14]:
# Train the model with GridSearch
best_model = grid.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 40 candidates, totalling 120 fits
[CV] C=0.0001, penalty=l1 ............................................
[CV] ................ C=0.0001, penalty=l1, score=0.241, total=   0.0s
[CV] C=0.0001, penalty=l1 ............................................
[CV] ................ C=0.0001, penalty=l1, score=0.241, total=   0.0s
[CV] C=0.0001, penalty=l1 ............................................
[CV] ................ C=0.0001, penalty=l1, score=0.241, total=   0.0s
[CV] C=0.0001, penalty=l2 ............................................
[CV] ................ C=0.0001, penalty=l2, score=0.501, total=   0.0s
[CV] C=0.0001, penalty=l2 ............................................
[CV] ................ C=0.0001, penalty=l2, score=0.501, total=   0.0s
[CV] C=0.0001, penalty=l2 ............................................
[CV] ................ C=0.0001, penalty=l2, score=0.501, total=   0.0s
[CV] C=0.00026366508987303583, penalty=l1 ............................
[CV]  C=0.00026

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] . C=0.0006951927961775605, penalty=l2, score=0.501, total=   0.0s
[CV] C=0.0006951927961775605, penalty=l2 .............................
[CV] . C=0.0006951927961775605, penalty=l2, score=0.501, total=   0.0s
[CV] C=0.0006951927961775605, penalty=l2 .............................
[CV] . C=0.0006951927961775605, penalty=l2, score=0.501, total=   0.0s
[CV] C=0.0018329807108324356, penalty=l1 .............................
[CV] . C=0.0018329807108324356, penalty=l1, score=0.501, total=   0.0s
[CV] C=0.0018329807108324356, penalty=l1 .............................
[CV] . C=0.0018329807108324356, penalty=l1, score=0.501, total=   0.0s
[CV] C=0.0018329807108324356, penalty=l1 .............................
[CV] . C=0.0018329807108324356, penalty=l1, score=0.501, total=   0.0s
[CV] C=0.0018329807108324356, penalty=l2 .............................
[CV] . C=0.0018329807108324356, penalty=l2, score=0.501, total=   0.0s
[CV] C=0.0018329807108324356, penalty=l2 .............................
[CV] .

[CV] .... C=11.288378916846883, penalty=l1, score=0.862, total=   0.4s
[CV] C=11.288378916846883, penalty=l1 ................................
[CV] .... C=11.288378916846883, penalty=l1, score=0.858, total=   0.7s
[CV] C=11.288378916846883, penalty=l2 ................................
[CV] .... C=11.288378916846883, penalty=l2, score=0.832, total=   0.0s
[CV] C=11.288378916846883, penalty=l2 ................................
[CV] .... C=11.288378916846883, penalty=l2, score=0.854, total=   0.0s
[CV] C=11.288378916846883, penalty=l2 ................................
[CV] .... C=11.288378916846883, penalty=l2, score=0.845, total=   0.0s
[CV] C=29.763514416313132, penalty=l1 ................................
[CV] .... C=29.763514416313132, penalty=l1, score=0.852, total=   0.6s
[CV] C=29.763514416313132, penalty=l1 ................................
[CV] .... C=29.763514416313132, penalty=l1, score=0.862, total=   0.4s
[CV] C=29.763514416313132, penalty=l1 ................................
[CV] .

[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:   41.9s finished


In [31]:
print(best_model.best_params_)

{'C': 3792.690190732246, 'penalty': 'l1'}


# Predict with Tuned Hyperparameters

In [32]:
# Predict using best parameters on scaled test set
grid_predictions = best_model.predict(X_test_scaled)
df_grid = pd.DataFrame({"Actual":y_test, "Predicted":grid_predictions}) 
df_grid.head()

,Actual,Predicted
3040,CANDIDATE,CONFIRMED
919,FALSE POSITIVE,FALSE POSITIVE
4307,FALSE POSITIVE,FALSE POSITIVE
5844,FALSE POSITIVE,FALSE POSITIVE
6734,FALSE POSITIVE,FALSE POSITIVE


In [33]:
print('Best Score: ', best_model.best_score_)

Best Score:  0.8632462330726683
